In [1]:
from bs4 import BeautifulSoup
import pandas as pd 
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
url_unique ='https://www.superga.com'
url_base = 'https://www.superga.com/collections/'
url_page = 'womens-ankle-boots'
url = url_base+url_page
header = {"User-Agent":"Chrome/39.0"}
print(url)
page = requests.get(url, headers=header)
print(page)

https://www.superga.com/collections/womens-ankle-boots
<Response [200]>


In [3]:
soup = BeautifulSoup(page.text)
stivaletti = soup.find_all('a', class_="grid-product__link")

print(stivaletti)


[<a class="grid-product__link" href="/collections/womens-ankle-boots/products/ankle-boots-woman-3051-alpina-apex-high-total-black-s1171qw-997">
<div class="grid-product__meta cb-text-left">
<div class="grid-product__title grid-product__title--heading">   3051 ALPINA APEX HIGH<span class="cb-italic cb-block cb-font-500">TOTAL BLACK</span></div><div class="grid-product__price">
<span class="">
  
      from
    
  
    €105,00
  
</span>
</div></div>
</a>, <a class="grid-product__link" href="/collections/womens-ankle-boots/products/ankle-boots-woman-3051-alpina-apex-high-total-beige-natural-s1171qw-ar3">
<div class="grid-product__meta cb-text-left">
<div class="grid-product__title grid-product__title--heading">   3051 ALPINA APEX HIGH<span class="cb-italic cb-block cb-font-500">TOTAL BEIGE NATURAL</span></div><div class="grid-product__price">
<span class="">
  
      from
    
  
    €105,00
  
</span>
</div></div>
</a>, <a class="grid-product__link" href="/collections/womens-ankle-boots

In [4]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode (no UI)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [5]:
rows = []
for product in stivaletti:
    try:
        # Get the product URL
        url_stivaletti = product.get('href')  # Get the link from ciabattenomi
        url1 = url_unique + url_stivaletti  # Construct the full URL
        driver.get(url1)  # Load the product page

        # Wait for JavaScript to render the page
        time.sleep(5)

        # Extract the rendered HTML
        html_content = driver.page_source
        soup_stivaletti = BeautifulSoup(html_content, 'html.parser')

        # Extract product details
        name_stivaletti = soup_stivaletti.find('h1', class_='h2 product-single__title')
        colore_stivaletti = soup_stivaletti.find('span', class_='cb-italic cb-block cb-font-500')
        prezzo_originale_stivaletti = soup_stivaletti.find('span', class_='related__price mr')
        colors_divs = soup_stivaletti.find_all('div', class_='related__color-name skeleton-effect')
        sizes = soup_stivaletti.find('ul', class_='related__select-optgroup-options')

        nomi = name_stivaletti.text.strip()
        colore =colore_stivaletti.text.strip()
        prezzo_originale = prezzo_originale_stivaletti.text.strip()
        
        # Remove color info from the name
        if colore in nomi:
            nomi = nomi.replace(colore, "")

        all_colors = []
        
        # Get all colors
        for color_div in colors_divs:
            color_name = color_div.text.strip()
            if color_name:  # Ensure the color name is not empty
                all_colors.append(color_name)

        # Get all sizes
        all_size = []

        for size in sizes :
            size_val = size.text.strip()
            if size :
                all_size.append(size_val)
        print(nomi)
        print(colore)
        print(prezzo_originale)
        print(all_colors)
        print(all_size)
        # Loop through all colors and sizes to add combinations to rows
        for color in all_colors:
            for size in all_size:
                rows.append({
                    'name': nomi,                 # Product name
                    'color': color,               # Color option
                    'size': size[:2],                 # Size option
                    'price_sale': "",                   # Sale price
                    'price': prezzo_originale[1:]      # Original price
                })
        print(rows)
    except Exception as e:
        print(f"Error processing product: {product}. Error: {e}")

# Create DataFrame from the rows
df = pd.DataFrame(rows)


# Display the DataFrame
print(df)

3051 ALPINA APEX HIGH
TOTAL BLACK
€105.00
['TOTAL BEIGE NATURAL', 'TOTAL BLACK']
['35', '36', '37', '38', '39', '40', '41', '42Available in other colors']
[{'name': '3051 ALPINA APEX HIGH', 'color': 'TOTAL BEIGE NATURAL', 'size': '35', 'price_sale': '', 'price': '105.00'}, {'name': '3051 ALPINA APEX HIGH', 'color': 'TOTAL BEIGE NATURAL', 'size': '36', 'price_sale': '', 'price': '105.00'}, {'name': '3051 ALPINA APEX HIGH', 'color': 'TOTAL BEIGE NATURAL', 'size': '37', 'price_sale': '', 'price': '105.00'}, {'name': '3051 ALPINA APEX HIGH', 'color': 'TOTAL BEIGE NATURAL', 'size': '38', 'price_sale': '', 'price': '105.00'}, {'name': '3051 ALPINA APEX HIGH', 'color': 'TOTAL BEIGE NATURAL', 'size': '39', 'price_sale': '', 'price': '105.00'}, {'name': '3051 ALPINA APEX HIGH', 'color': 'TOTAL BEIGE NATURAL', 'size': '40', 'price_sale': '', 'price': '105.00'}, {'name': '3051 ALPINA APEX HIGH', 'color': 'TOTAL BEIGE NATURAL', 'size': '41', 'price_sale': '', 'price': '105.00'}, {'name': '3051 ALP

In [6]:
# Save the DataFrame to a CSV file
df.to_csv('db_stivaletti.csv', index=False)